```
note:

using the table of contents panel makes this notebook easier to navigate
all of my written analysis is done in the "Write Up" section of the resepctive portion of the project
```

# Part 1

## Imports

In [ ]:
from __future__ import print_function
import argparse
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
# from ConvNet import ConvNet 
import argparse
import numpy as np 

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## ConvNet

create the three models

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, mode):
        super(ConvNet, self).__init__()

        #model 0
        self.m0_fc1 = nn.Linear(28*28, 100).to(device)
        self.m0_fc2 = nn.Linear(100, 100).to(device)
        self.m0_fc3 = nn.Linear(100, 10).to(device)

        #model 1
        self.m1_fc1 = nn.Linear(28*28, 20).to(device) # 20 neurons
        self.m1_fc2 = nn.Linear(20, 20).to(device) # 20 neurons
        self.m1_fc3 = nn.Linear(20, 10).to(device) # 10 for MINST

        #model 2
        self.m2_fc1 = nn.Linear(28*28, 20).to(device) # 20 neurons
        self.m2_fc2 = nn.Linear(20, 20).to(device) # 20 neurons
        self.m2_fc3 = nn.Linear(20, 10).to(device) # 10 for MINST

        #model 3
        self.m3_fc1 = nn.Linear(28*28, 2048).to(device)
        self.m3_fc2 = nn.Linear(2048, 1024).to(device)
        self.m3_fc3 = nn.Linear(1024, 512).to(device)
        self.m3_fc4 = nn.Linear(512, 256).to(device)
        self.m3_fc5 = nn.Linear(256, 10).to(device)      
        # self.m3_fc1 = nn.Linear(28*28, 1024).to(device)
        # self.m3_fc2 = nn.Linear(1024, 512).to(device)
        # self.m3_fc3 = nn.Linear(512, 256).to(device)
        # self.m3_fc4 = nn.Linear(256, 128).to(device)
        # self.m3_fc5 = nn.Linear(128, 64).to(device)      
        # self.m3_fc6 = nn.Linear(64, 32).to(device)
        # self.m3_fc7 = nn.Linear(32, 16).to(device)
        # self.m3_fc8 = nn.Linear(16, 10).to(device)
             
        # This will select the forward pass function based on mode for the ConvNet.
        # During creation of each ConvNet model, you will assign one of the valid mode.
        # This will fix the forward function (and the network graph) for the entire training/testing
        if mode == 1:
            self.forward = self.model_1
        elif mode == 2:
            self.forward = self.model_2
        elif mode == 3:
            self.forward = self.model_3
        else: 
            print("Invalid mode ", mode, "selected. Select between 1-3")
            exit(0)
      
    # Baseline sample model
    def model_0(self, X):
        # ======================================================================
        # Three fully connected layers with activation
        
        X = torch.flatten(X, start_dim=1)
        X = self.m0_fc1(X)
        X = F.relu(X)
        X = self.m0_fc2(X)
        X = F.relu(X)
        X = self.m0_fc3(X)
        X = torch.sigmoid(X)
                
        return X  
        
    # Baseline model. task 1
    def model_1(self, X):
        # ======================================================================
        # Three fully connected layers without activation

        # print('1 ')

        X = X.to(device)
        X = torch.flatten(X, start_dim=1)

        X = self.m1_fc1(X)
        X = self.m1_fc2(X)
        X = self.m1_fc3(X)

        return X
        

    # task 2
    def model_2(self, X):
        # ======================================================================
        # Train with activation (use model 1 from task 1)

        # print('2 ')
        
        X = X.to(device)
        X = torch.flatten(X, start_dim=1)

        X = self.m2_fc1(X)
        X = F.relu(X)
        X = self.m2_fc2(X)
        X = F.relu(X)
        X = self.m2_fc3(X)
        X = F.relu(X)

        # X = torch.sigmoid(X) # i don't think i can use this cause the instructions say to only use RELU activation
        
        return X

	
    # task 3
    def model_3(self, X):
        # ======================================================================
        # Change number of fully connected layers and number of neurons from model 2 in task 2

        # print('3 ')
        
        X = X.to(device)
        X = torch.flatten(X, start_dim=1)

        X = self.m3_fc1(X)
        X = F.relu(X)
        X = self.m3_fc2(X)
        X = F.relu(X)
        X = self.m3_fc3(X)
        X = F.relu(X)
        X = self.m3_fc4(X)
        X = F.relu(X)
        X = self.m3_fc5(X)
        # these are old layers made before i realized we needed >200 neurons
        # X = F.relu(X)
        # X = self.m3_fc6(X)
        # X = F.relu(X)
        # X = self.m3_fc7(X)
        # X = F.relu(X)
        # X = self.m3_fc8(X)

        X = F.relu(X) # i think i have to use this?
        # X = torch.sigmoid(X) # commented out b/c i wanted to try softmax
        # X = F.log_softmax(X, dim=1)
        
        return X
        

## Helpers

these functions were provided by the professor, and weren't modified a lot

### Train

In [ ]:
def train(model, device, train_loader, optimizer, criterion, epoch, batch_size):
    '''
    Trains the model for an epoch and optimizes it.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    train_loader: dataloader for training samples.
    optimizer: optimizer to use for model parameter updates.
    criterion: used to compute loss for prediction and target 
    epoch: Current epoch to train for.
    batch_size: Batch size to be used.
    '''
    
    # Set model to train mode before each epoch
    model.train()
    
    # Empty list to store losses 
    losses = []
    correct = 0
    
    # Iterate over entire training samples (1 epoch)
    for batch_idx, batch_sample in enumerate(train_loader):
        data, target = batch_sample
        
        # Push data/label to correct device
        data, target = data.to(device), target.to(device)
        
        # Reset optimizer gradients. Avoids grad accumulation (accumulation used in RNN).
        optimizer.zero_grad()
        
        # Do forward pass for current set of data
        output = model(data)
        
        # ======================================================================
        # Compute loss based on criterion
        # ----------------- YOUR CODE HERE ----------------------
        loss = criterion(output, target)
        
        # Computes gradient based on final loss
        loss.backward()
        
        # Store loss
        losses.append(loss.item())
        
        # Optimize model parameters based on learning rate and gradient 
        optimizer.step()
        
        # Get predicted index by selecting maximum log-probability
        pred = output.argmax(dim=1, keepdim=True)
        
        # ======================================================================
        # Count correct predictions overall 
        correct += pred.eq(target.view_as(pred)).sum().item()
        
    train_loss = float(np.mean(losses))
    train_acc = correct / ((batch_idx+1) * batch_size)
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        float(np.mean(losses)), correct, (batch_idx+1) * batch_size,
        100. * correct / ((batch_idx+1) * batch_size)))
    return train_loss, train_acc
    

### Test

In [ ]:
def test(model, device, test_loader):
    '''
    Tests the model.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    test_loader: dataloader for test samples.
    '''
    
    # Set model to eval mode to notify all layers.
    model.eval()
    
    losses = []
    correct = 0
    
    # Set torch.no_grad() to disable gradient computation and backpropagation
    with torch.no_grad():
        for batch_idx, sample in enumerate(test_loader):
            data, target = sample
            data, target = data.to(device), target.to(device)
            

            # Predict for data by doing forward pass
            output = model(data)
            
            # ======================================================================
            # Compute loss based on same criterion as training
            loss = F.cross_entropy(output, target, reduction='mean')
            
            # Append loss to overall test loss
            losses.append(loss.item())
            
            # Get predicted index by selecting maximum log-probability
            pred = output.argmax(dim=1, keepdim=True)
            
            # ======================================================================
            # Count correct predictions overall 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss = float(np.mean(losses))
    accuracy = 100. * correct / len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    
    return test_loss, accuracy

### Main

In [ ]:
def run_main(FLAGS):
    # # Check if cuda is available
    # use_cuda = torch.cuda.is_available()
    
    # # Set proper device based on cuda availability 
    # # device = torch.device("cuda" if use_cuda else "cpu")
    # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Torch device selected: ", device)
    
    # Initialize the model and send to device 
    model = ConvNet(FLAGS.mode).to(device)
    
    # Initialize the criterion for loss computation 
    # ======================================================================
    criterion = torch.nn.CrossEntropyLoss()
    
    # Initialize optimizer type 
    optimizer = optim.SGD(model.parameters(), lr=FLAGS.learning_rate, weight_decay=1e-7)
    
    # Create transformations to apply to each data sample 
    # Can specify variations such as image flip, color flip, random crop, ...
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    
    # Load datasets for training and testing
    # Inbuilt datasets available in torchvision (check documentation online)
    dataset1 = datasets.MNIST('./data/', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('./data/', train=False,
                       transform=transform)
    train_loader = DataLoader(dataset1, batch_size = FLAGS.batch_size, 
                                shuffle=True, num_workers=2)#4)
    test_loader = DataLoader(dataset2, batch_size = FLAGS.batch_size, 
                                shuffle=False, num_workers=2)#4)
    
    best_accuracy = 0.0
    
    # Run training for n_epochs specified in config 
    for epoch in range(1, FLAGS.num_epochs + 1):
        print("\nEpoch: ", epoch)
        train_loss, train_accuracy = train(model, device, train_loader,
                                            optimizer, criterion, epoch, FLAGS.batch_size)
        
        test_loss, test_accuracy = test(model, device, test_loader)
        
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            
    print("\nBest accuracy is {:2.2f}".format(best_accuracy))
    
    print("\nTraining and evaluation finished")

## RUN

here we run the 3 models we created throughout completing the tasks

### Model 1

#### Code

In [ ]:
# Set parameters for Sparse Autoencoder
parser = argparse.ArgumentParser('CNN Exercise.')
parser.add_argument('--mode',
                    type=int, default=1,
                    help='Select mode between 1-3.')
parser.add_argument('--learning_rate',
                    type=float, default=0.1,
                    help='Initial learning rate.')
parser.add_argument('--num_epochs',
                    type=int,
                    default=5,
                    help='Number of epochs to run trainer.')
parser.add_argument('--batch_size',
                    type=int, default=10,
                    help='Batch size. Must divide evenly into the dataset sizes.')
parser.add_argument('--log_dir',
                    type=str,
                    default='logs',
                    help='Directory to put logging.')
                    
FLAGS = None
FLAGS, unparsed = parser.parse_known_args()

print("Mode: ", FLAGS.mode)
print("LR: ", FLAGS.learning_rate)
print("Batch size: ", FLAGS.batch_size)

start_time = time.time()
run_main(FLAGS) 
end_time = time.time()

print(f'\nTook: {end_time - start_time: .2f} seconds')

Mode:  1
LR:  0.1
Batch size:  10
Torch device selected:  cuda:0
Files already downloaded and verified

Epoch:  1
Train set: Average loss: 1.6339, Accuracy: 20350/50000 (41%)
Test set: Average loss: 1.4817, Accuracy: 4646/10000 (46%)

Epoch:  2
Train set: Average loss: 1.4414, Accuracy: 24399/50000 (49%)
Test set: Average loss: 1.3998, Accuracy: 5113/10000 (51%)

Epoch:  3
Train set: Average loss: 1.3860, Accuracy: 25677/50000 (51%)
Test set: Average loss: 1.7159, Accuracy: 4188/10000 (42%)

Epoch:  4
Train set: Average loss: 1.3422, Accuracy: 26505/50000 (53%)
Test set: Average loss: 1.4264, Accuracy: 5198/10000 (52%)

Epoch:  5
Train set: Average loss: 1.3262, Accuracy: 26880/50000 (54%)
Test set: Average loss: 1.3930, Accuracy: 5275/10000 (53%)

Best accuracy is 52.75

Training and evaluation finished

Took:  173.29 seconds


#### Previous Log

```
Mode:  1
LR:  0.1
Batch size:  10
Torch device selected:  cuda:0

Epoch:  1
Train set: Average loss: nan, Accuracy: 27545/60000 (46%)
Test set: Average loss: nan, Accuracy: 980/10000 (10%)

Epoch:  2
Train set: Average loss: nan, Accuracy: 5923/60000 (10%)
Test set: Average loss: nan, Accuracy: 980/10000 (10%)

Epoch:  3
Train set: Average loss: nan, Accuracy: 5923/60000 (10%)
Test set: Average loss: nan, Accuracy: 980/10000 (10%)

Epoch:  4
Train set: Average loss: nan, Accuracy: 5923/60000 (10%)
Test set: Average loss: nan, Accuracy: 980/10000 (10%)

Epoch:  5
Train set: Average loss: nan, Accuracy: 5923/60000 (10%)
Test set: Average loss: nan, Accuracy: 980/10000 (10%)

Best accuracy is 9.80

Training and evaluation finished

Took:  174.70 seconds
```

### Model 2

#### Code

In [ ]:
# Set parameters for Sparse Autoencoder
parser = argparse.ArgumentParser('CNN Exercise.')
parser.add_argument('--mode',
                    type=int, default=2,
                    help='Select mode between 1-3.')
parser.add_argument('--learning_rate',
                    type=float, default=0.1,
                    help='Initial learning rate.')
parser.add_argument('--num_epochs',
                    type=int,
                    default=5,
                    help='Number of epochs to run trainer.')
parser.add_argument('--batch_size',
                    type=int, default=10,
                    help='Batch size. Must divide evenly into the dataset sizes.')
parser.add_argument('--log_dir',
                    type=str,
                    default='logs',
                    help='Directory to put logging.')
                    
FLAGS = None
FLAGS, unparsed = parser.parse_known_args()

print("Mode: ", FLAGS.mode)
print("LR: ", FLAGS.learning_rate)
print("Batch size: ", FLAGS.batch_size)

start_time = time.time()
run_main(FLAGS)
end_time = time.time()

print(f'\nTook: {end_time - start_time: .2f} seconds')

Mode:  2
LR:  0.1
Batch size:  10
Torch device selected:  cuda:0

Epoch:  1
Train set: Average loss: 0.3499, Accuracy: 53868/60000 (90%)
Test set: Average loss: 0.4477, Accuracy: 8688/10000 (87%)

Epoch:  2
Train set: Average loss: 0.2420, Accuracy: 55997/60000 (93%)
Test set: Average loss: 0.2065, Accuracy: 9435/10000 (94%)

Epoch:  3
Train set: Average loss: 0.2137, Accuracy: 56558/60000 (94%)
Test set: Average loss: 0.2099, Accuracy: 9420/10000 (94%)

Epoch:  4
Train set: Average loss: 0.2033, Accuracy: 56711/60000 (95%)
Test set: Average loss: 0.1685, Accuracy: 9551/10000 (96%)

Epoch:  5
Train set: Average loss: 0.1969, Accuracy: 56838/60000 (95%)
Test set: Average loss: 0.2199, Accuracy: 9464/10000 (95%)

Best accuracy is 95.51

Training and evaluation finished

Took:  186.33 seconds


#### Previous Log

```
Mode:  2
LR:  0.1
Batch size:  10
Torch device selected:  cuda:0

Epoch:  1
Train set: Average loss: 1.5871, Accuracy: 50087/60000 (83%)
Test set: Average loss: 1.5339, Accuracy: 9150/10000 (92%)

Epoch:  2
Train set: Average loss: 1.5242, Accuracy: 55597/60000 (93%)
Test set: Average loss: 1.5138, Accuracy: 9394/10000 (94%)

Epoch:  3
Train set: Average loss: 1.5130, Accuracy: 56345/60000 (94%)
Test set: Average loss: 1.5082, Accuracy: 9433/10000 (94%)

Epoch:  4
Train set: Average loss: 1.5076, Accuracy: 56647/60000 (94%)
Test set: Average loss: 1.5094, Accuracy: 9441/10000 (94%)

Epoch:  5
Train set: Average loss: 1.5045, Accuracy: 56895/60000 (95%)
Test set: Average loss: 1.5052, Accuracy: 9471/10000 (95%)

Best accuracy is 94.71

Training and evaluation finished

Took:  178.59 seconds
```

### Model 3

#### Code

In [ ]:
# Set parameters for Sparse Autoencoder
parser = argparse.ArgumentParser('CNN Exercise.')
parser.add_argument('--mode',
                    type=int, default=3,
                    help='Select mode between 1-3.')
parser.add_argument('--learning_rate',
                    type=float, default=0.1,
                    help='Initial learning rate.')
parser.add_argument('--num_epochs',
                    type=int,
                    default=5,
                    help='Number of epochs to run trainer.')
parser.add_argument('--batch_size',
                    type=int, default=10,
                    help='Batch size. Must divide evenly into the dataset sizes.')
parser.add_argument('--log_dir',
                    type=str,
                    default='logs',
                    help='Directory to put logging.')
                    
FLAGS = None
FLAGS, unparsed = parser.parse_known_args()

print("Mode: ", FLAGS.mode)
print("LR: ", FLAGS.learning_rate)
print("Batch size: ", FLAGS.batch_size)

start_time = time.time()
run_main(FLAGS)
end_time = time.time()

print(f'\nTook: {end_time - start_time: .2f} seconds')

Mode:  3
LR:  0.1
Batch size:  10
Torch device selected:  cuda:0

Epoch:  1
Train set: Average loss: 0.2509, Accuracy: 55551/60000 (93%)
Test set: Average loss: 0.1194, Accuracy: 9662/10000 (97%)

Epoch:  2
Train set: Average loss: 0.1004, Accuracy: 58241/60000 (97%)
Test set: Average loss: 0.0896, Accuracy: 9738/10000 (97%)

Epoch:  3
Train set: Average loss: 0.0637, Accuracy: 58904/60000 (98%)
Test set: Average loss: 0.0889, Accuracy: 9760/10000 (98%)

Epoch:  4
Train set: Average loss: 0.0479, Accuracy: 59115/60000 (99%)
Test set: Average loss: 0.0812, Accuracy: 9790/10000 (98%)

Epoch:  5
Train set: Average loss: 0.0373, Accuracy: 59357/60000 (99%)
Test set: Average loss: 0.0824, Accuracy: 9769/10000 (98%)

Best accuracy is 97.90

Training and evaluation finished

Took:  199.02 seconds


#### Previous Log

```
Mode:  3
LR:  0.1
Batch size:  10
Torch device selected:  cuda:0

Epoch:  1
Train set: Average loss: 0.2522, Accuracy: 55447/60000 (92%)
Test set: Average loss: 0.1450, Accuracy: 9562/10000 (96%)

Epoch:  2
Train set: Average loss: 0.0975, Accuracy: 58285/60000 (97%)
Test set: Average loss: 0.0900, Accuracy: 9728/10000 (97%)

Epoch:  3
Train set: Average loss: 0.0678, Accuracy: 58797/60000 (98%)
Test set: Average loss: 0.0811, Accuracy: 9771/10000 (98%)

Epoch:  4
Train set: Average loss: 0.0473, Accuracy: 59155/60000 (99%)
Test set: Average loss: 0.0884, Accuracy: 9769/10000 (98%)

Epoch:  5
Train set: Average loss: 0.0352, Accuracy: 59353/60000 (99%)
Test set: Average loss: 0.0690, Accuracy: 9817/10000 (98%)

Best accuracy is 98.17

Training and evaluation finished

Took:  199.08 seconds
```

#### Reminder

In [ ]:
'''

NOTE: at this point i didnt realize the instructions said each layer HAS to have >200 neurons 

def example_of_old_model3(): 
    
    self.m3_fc1 = nn.Linear(28*28, 1024)
    self.m3_bn1 = nn.BatchNorm1d(1024)
    self.m3_dropout1 = nn.Dropout(p=0.5)
    self.m3_fc2 = nn.Linear(1024, 512)
    self.m3_bn2 = nn.BatchNorm1d(512)
    self.m3_dropout2 = nn.Dropout(p=0.5)
    self.m3_fc3 = nn.Linear(512, 256)
    self.m3_bn3 = nn.BatchNorm1d(256)
    self.m3_dropout3 = nn.Dropout(p=0.5)
    self.m3_fc4 = nn.Linear(256, 128)
    self.m3_bn4 = nn.BatchNorm1d(128)
    self.m3_dropout4 = nn.Dropout(p=0.5)
    self.m3_fc5 = nn.Linear(128, 64)
    self.m3_bn5 = nn.BatchNorm1d(64)
    self.m3_dropout5 = nn.Dropout(p=0.5)
    self.m3_fc6 = nn.Linear(64, 32)
    self.m3_bn6 = nn.BatchNorm1d(32)
    self.m3_dropout6 = nn.Dropout(p=0.5)
    self.m3_fc7 = nn.Linear(32, 16)
    self.m3_bn7 = nn.BatchNorm1d(16)
    self.m3_dropout7 = nn.Dropout(p=0.5)
    self.m3_fc8 = nn.Linear(16, 10)

        model_3()

    def model_3 (self, X):
        
        X = F.relu(self.m3_bn1(self.m3_fc1(X)))
        X = self.m3_dropout1(X)
        X = F.relu(self.m3_bn2(self.m3_fc2(X)))
        X = self.m3_dropout2(X)
        X = F.relu(self.m3_bn3(self.m3_fc3(X)))
        X = self.m3_dropout3(X)
        X = F.relu(self.m3_bn4(self.m3_fc4(X)))
        X = self.m3_dropout4(X)
        X = F.relu(self.m3_bn5(self.m3_fc5(X)))
        X = self.m3_dropout5(X)
        X = F.relu(self.m3_bn6(self.m3_fc6(X)))
        X = self.m3_dropout6(X)
        X = F.relu(self.m3_bn7(self.m3_fc7(X)))
        X = self.m3_dropout7(X)
        X = self.m3_fc8(X)
    
        X = F.log_softmax(X, dim=1)
'''

# this model requires a lot of epochs

'\n\nNOTE: at this point i didnt realize the instructions said each layer HAS to have >200 neurons \n\ndef example_of_old_model3(): \n    \n    self.m3_fc1 = nn.Linear(28*28, 1024)\n    self.m3_bn1 = nn.BatchNorm1d(1024)\n    self.m3_dropout1 = nn.Dropout(p=0.5)\n    self.m3_fc2 = nn.Linear(1024, 512)\n    self.m3_bn2 = nn.BatchNorm1d(512)\n    self.m3_dropout2 = nn.Dropout(p=0.5)\n    self.m3_fc3 = nn.Linear(512, 256)\n    self.m3_bn3 = nn.BatchNorm1d(256)\n    self.m3_dropout3 = nn.Dropout(p=0.5)\n    self.m3_fc4 = nn.Linear(256, 128)\n    self.m3_bn4 = nn.BatchNorm1d(128)\n    self.m3_dropout4 = nn.Dropout(p=0.5)\n    self.m3_fc5 = nn.Linear(128, 64)\n    self.m3_bn5 = nn.BatchNorm1d(64)\n    self.m3_dropout5 = nn.Dropout(p=0.5)\n    self.m3_fc6 = nn.Linear(64, 32)\n    self.m3_bn6 = nn.BatchNorm1d(32)\n    self.m3_dropout6 = nn.Dropout(p=0.5)\n    self.m3_fc7 = nn.Linear(32, 16)\n    self.m3_bn7 = nn.BatchNorm1d(16)\n    self.m3_dropout7 = nn.Dropout(p=0.5)\n    self.m3_fc8 = nn.Li

## Write Up

### Task 1

I was uncertain of my results at first. However, after doing research, my results seem to be a good sign. 

The best accuracy was 9.80

In model 1 we can see nan values, thus all the statistical data we "collected" is useless. My research indicates that a model without any activation functions the model won't ve able to learn effectively. As a result, the average loss will become nan due to numerical instability. The model may have become too large or too small, which can cause floating-point values to overflow. 

That being said, it continued to try and train for as long as I said to and took approx. 174 seconds to complete.


### Task 2

With the inclusion of activation function we can see a stable model being trained. The results speak for themselves. The highest accuracy was around 95%  w/ a loss hovering slightly above 1.5 (this applies for both the training and test set). 

This loss is still pretty high, indicating that it is making some errors. And, since the loss never really changed, it's possible that it became stuck (plateaued) on a local minimum. It's possible that changing the hyper parameters would lead to a better result, but I wanted to keep it consistent throughout the models to see where the performance varies. 

So, for a thought experiment let's assume the hyperparameters were not the issue. The data is obviously fine, so poor data quality cannot be the issue. Overfitting is unlikely because both the training and test set have poor performance. With these ideas out of the way, it becomes conclusive that there's poor model capacity. Meaning, the model was not powerful enough to learn the patterns of the data (underfitting). Since it's unedrfitting adding more neurons and more layers should result in a bump in performance.

This model took around 180 seconds, which is only a few more than model 1. 

### Task 3

Here the model performed exceptionally well. It's hard to say whether or not it's overfitting based on this alone, but I'll explain why this might not be the case. 

The best accuracy is 99% for training and 98% for test.

Here we see a strong accuracy after the first epoch with consistent improvements as it progresses. The loss however drops a lot, which is generally a good thing. But it's important to note that it's really only a good thing if both the training and test set have low loss, this is the case for this particular model. 

I will note however that the loss is SLIGHTLY larger for the test set. However, much lower on the first epoch. I believe it's because the model is still learning and as a result able to improve quickly. Online I found that this was a good sign, as it points to my model fitting the data appropriately (neither over- or underfitting).

This model took a more time than the original two, but not much more, at approxtimately 200 seconds. 

# Part 2

## Imports

In [ ]:
from __future__ import print_function
import argparse
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
#from ConvNet import ConvNet 
import argparse
import numpy as np 

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

## CNN

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, mode):
        super(ConvNet, self).__init__()
        
        # Define various layers here, such as in the tutorial example
        self.conv1   = nn.Conv2d(in_channels=3,  out_channels=10, kernel_size=3)
        self.conv2   = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3)
        self.conv3   = nn.Conv2d(in_channels=20, out_channels=40, kernel_size=3)
        self.conv1_1 = nn.Conv2d(in_channels=3,  out_channels=20, kernel_size=3)
        self.conv3_1 = nn.Conv2d(in_channels=40, out_channels=40, kernel_size=3)
        
        
        self.fc1_model1 = nn.Linear(360, 100)  # This is first fully connected layer for step 1.
        self.fc1_model2 = nn.Linear(1440, 100) # This is first fully connected layer for step 2.
        self.fc1_model3 = nn.Linear(640, 100)  # This is first fully connected layer for step 3
        
        self.fc2 = nn.Linear(100, 10)          # This is 2nd fully connected layer for all models.
        
        self.fc_model0 = nn.Linear(2250, 100)  # This is for example model.
        
        
        # This will select the forward pass function based on mode for the ConvNet.
        # Based on the question, you have 3 modes available for step 1 to 3.
        # During creation of each ConvNet model, you will assign one of the valid mode.
        # This will fix the forward function (and the network graph) for the entire training/testing
        if mode == 1:
            self.forward = self.model_1
        elif mode == 2:
            self.forward = self.model_2
        elif mode == 3:
            self.forward = self.model_3
        elif mode == 0:
            self.forward = self.model_0
        else: 
            print("Invalid mode ", mode, "selected. Select between 1-3")
            exit(0)
        
    
    # Example model. Modify this for step 1-3
    def model_0(self, X):
        # ======================================================================         
        
        X = F.relu(self.conv1(X))
        #print(X.shape)
        X = F.max_pool2d(X, kernel_size=2)
        #print(X.shape)
        
        X = torch.flatten(X, start_dim=1)
        #print(X.shape)
        
        X = F.relu(self.fc_model0(X))
        X = self.fc2(X)
        
        return X
        
    
    # Simple CNN. step 1
    def model_1(self, X):
        # ======================================================================
         
        # Complete this part as model_0, add one more conv2d layer 
        # with relu activation followed by maxpool layer.
        
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, kernel_size=2)

        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, kernel_size=2)

        X = torch.flatten(X, start_dim=1)

        X = F.relu(self.fc1_model1(X))
        X = self.fc2(X)
        
        return X
        

    # Increase filters. step 2
    def model_2(self, X):
        # ======================================================================
        
        # Complete this part as model_1. Modify in/out channels for conv2d layers.
        
        X = F.relu(self.conv1_1(X))         # 3 in,  20 out
        X = F.max_pool2d(X, kernel_size=2)

        X = F.relu(self.conv3(X))           # 20 in, 40 out
        X = F.max_pool2d(X, kernel_size=2)

        X = torch.flatten(X, start_dim=1)

        X = F.relu(self.fc1_model2(X))
        X = self.fc2(X)
        
        return X
        

    # Large CNN. step 3
    def model_3(self, X):
        # ======================================================================
        
        # Complete this part as model_2, add one more conv2d layer 
        # with relu activation. Do not add maxpool after this new conv2d layer.

        X = F.relu(self.conv1_1(X))         # 3 in,  20 out
        X = F.max_pool2d(X, kernel_size=2)

        X = F.relu(self.conv3(X))           # 20 in,  40 out
        X = F.max_pool2d(X, kernel_size=2)

        X = F.relu(self.conv3_1(X))         # 40 in,  40 out
        
        X = torch.flatten(X, start_dim=1)

        X = F.relu(self.fc1_model3(X))
        X = self.fc2(X)
        
        return X

## Helpers

### Train

In [ ]:
def train(model, device, train_loader, optimizer, criterion, epoch, batch_size):
    '''
    Trains the model for an epoch and optimizes it.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    train_loader: dataloader for training samples.
    optimizer: optimizer to use for model parameter updates.
    criterion: used to compute loss for prediction and target 
    epoch: Current epoch to train for.
    batch_size: Batch size to be used.
    '''
    
    # Set model to train mode before each epoch
    model.train()
    
    # Empty list to store losses 
    losses = []
    correct = 0
    
    # Iterate over entire training samples (1 epoch)
    for batch_idx, batch_sample in enumerate(train_loader):
        data, target = batch_sample
        
        # Push data/label to correct device
        data, target = data.to(device), target.to(device)
        # print(data.shape)
        # print(target.shape)
        # exit()
        
        # Reset optimizer gradients. Avoids grad accumulation (accumulation used in RNN).
        optimizer.zero_grad()
        
        # Do forward pass for current set of data
        output = model(data)
        
        # ======================================================================
        # Compute loss based on criterion
        loss = criterion(output, target)
        
        # Computes gradient based on final loss
        loss.backward()
        
        # Store loss
        losses.append(loss.item())
        
        # Optimize model parameters based on learning rate and gradient 
        optimizer.step()
        
        # Get predicted index by selecting maximum log-probability
        pred = output.argmax(dim=1, keepdim=True)
        
        # ======================================================================
        # Count correct predictions overall 
        correct += pred.eq(target.view_as(pred)).sum().item()
        
    train_loss = float(np.mean(losses))
    train_acc = correct / ((batch_idx+1) * batch_size)
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        float(np.mean(losses)), correct, (batch_idx+1) * batch_size,
        100. * correct / ((batch_idx+1) * batch_size)))
    return train_loss, train_acc

### Test

In [ ]:
def test(model, device, test_loader):
    '''
    Tests the model.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    test_loader: dataloader for test samples.
    '''
    
    # Set model to eval mode to notify all layers.
    model.eval()
    
    losses = []
    correct = 0
    
    # Set torch.no_grad() to disable gradient computation and backpropagation
    with torch.no_grad():
        for batch_idx, sample in enumerate(test_loader):
            data, target = sample
            data, target = data.to(device), target.to(device)
            

            # Predict for data by doing forward pass
            output = model(data)
            
            # ======================================================================
            # Compute loss based on same criterion as training
            loss = F.cross_entropy(output, target, reduction='mean')
            
            # Append loss to overall test loss
            losses.append(loss.item())
            
            # Get predicted index by selecting maximum log-probability
            pred = output.argmax(dim=1, keepdim=True)
            
            # ======================================================================
            # Count correct predictions overall 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss = float(np.mean(losses))
    accuracy = 100. * correct / len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    
    return test_loss, accuracy

### Main

In [ ]:
def run_main(FLAGS):
    # # Check if cuda is available
    # use_cuda = torch.cuda.is_available()
    
    # # Set proper device based on cuda availability 
    # device = torch.device("cuda:0" if use_cuda else "cpu")
    print("Torch device selected: ", device)
    
    # Initialize the model and send to device 
    model = ConvNet(FLAGS.mode).to(device)
    # print(model)
    # exit()

    # Initialize the criterion for loss computation 
    criterion = nn.CrossEntropyLoss(reduction='mean')
    
    # Initialize optimizer type 
    optimizer = optim.SGD(model.parameters(), lr=FLAGS.learning_rate, weight_decay=1e-7)
    
    # Create transformations to apply to each data sample 
    # Can specify variations such as image flip, color flip, random crop, ...
    #transform=transforms.Compose([
    #    transforms.ToTensor(),
    #    transforms.Normalize((0.1307,), (0.3081,))
    #    ])
    
    transform = transforms.Compose(
                    [transforms.ToTensor(),
                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
     
    # Load datasets for training and testing
    # Inbuilt datasets available in torchvision (check documentation online)
    dataset1 = datasets.CIFAR10('./data/', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.CIFAR10('./data/', train=False,
                       transform=transform)
    train_loader = DataLoader(dataset1, batch_size = FLAGS.batch_size, 
                                shuffle=True, num_workers=4)
    test_loader = DataLoader(dataset2, batch_size = FLAGS.batch_size, 
                                shuffle=False, num_workers=4)
    
    best_accuracy = 0.0
    
    # Run training for n_epochs specified in config 
    for epoch in range(1, FLAGS.num_epochs + 1):
        print("\nEpoch: ", epoch)
        train_loss, train_accuracy = train(model, device, train_loader,
                                            optimizer, criterion, epoch, FLAGS.batch_size)
        test_loss, test_accuracy = test(model, device, test_loader)
        
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            
    print("\nBest accuracy is {:2.2f}".format(best_accuracy))
    
    print("\nTraining and evaluation finished")

## RUN

### Model 0 (test)

#### Code

In [ ]:
# Set parameters for Sparse Autoencoder
parser = argparse.ArgumentParser('CNN Exercise.')
parser.add_argument('--mode',
                    type=int, default=0,
                    help='Select mode between 1-3.')
parser.add_argument('--learning_rate',
                    type=float, default=0.1,
                    help='Initial learning rate.')
parser.add_argument('--num_epochs',
                    type=int,
                    default=10,
                    help='Number of epochs to run trainer.')
parser.add_argument('--batch_size',
                    type=int, default=100,
                    help='Batch size. Must divide evenly into the dataset sizes.')
parser.add_argument('--log_dir',
                    type=str,
                    default='logs',
                    help='Directory to put logging.')
                    
FLAGS = None
FLAGS, unparsed = parser.parse_known_args()

print("Mode: ", FLAGS.mode)
print("LR: ", FLAGS.learning_rate)
print("Batch size: ", FLAGS.batch_size)

start_time = time.time()
run_main(FLAGS)
end_time = time.time()

print(f'\nTook: {end_time - start_time: .2f} seconds')

Mode:  0
LR:  0.1
Batch size:  100
Torch device selected:  cuda:0
Files already downloaded and verified

Epoch:  1
Train set: Average loss: 1.6728, Accuracy: 19844/50000 (40%)
Test set: Average loss: 1.4185, Accuracy: 4836/10000 (48%)

Epoch:  2
Train set: Average loss: 1.3166, Accuracy: 26502/50000 (53%)
Test set: Average loss: 1.3185, Accuracy: 5265/10000 (53%)

Epoch:  3
Train set: Average loss: 1.1821, Accuracy: 29057/50000 (58%)
Test set: Average loss: 1.1938, Accuracy: 5763/10000 (58%)

Epoch:  4
Train set: Average loss: 1.0949, Accuracy: 30685/50000 (61%)
Test set: Average loss: 1.1699, Accuracy: 5900/10000 (59%)

Epoch:  5
Train set: Average loss: 1.0194, Accuracy: 31954/50000 (64%)
Test set: Average loss: 1.1829, Accuracy: 5858/10000 (59%)

Epoch:  6
Train set: Average loss: 0.9529, Accuracy: 33288/50000 (67%)
Test set: Average loss: 1.1074, Accuracy: 6112/10000 (61%)

Epoch:  7
Train set: Average loss: 0.8851, Accuracy: 34339/50000 (69%)
Test set: Average loss: 1.1352, Accura

#### Previous Log:

```
Mode:  0
LR:  0.1
Batch size:  100
Torch device selected:  cuda:0
Files already downloaded and verified

Epoch:  1
Train set: Average loss: 1.6622, Accuracy: 20273/50000 (41%)
Test set: Average loss: 1.3655, Accuracy: 5092/10000 (51%)

Epoch:  2
Train set: Average loss: 1.3129, Accuracy: 26563/50000 (53%)
Test set: Average loss: 1.2876, Accuracy: 5425/10000 (54%)

Epoch:  3
Train set: Average loss: 1.1766, Accuracy: 29110/50000 (58%)
Test set: Average loss: 1.2049, Accuracy: 5663/10000 (57%)

Epoch:  4
Train set: Average loss: 1.0763, Accuracy: 31024/50000 (62%)
Test set: Average loss: 1.1467, Accuracy: 5915/10000 (59%)

Epoch:  5
Train set: Average loss: 0.9956, Accuracy: 32430/50000 (65%)
Test set: Average loss: 1.1731, Accuracy: 5924/10000 (59%)

Epoch:  6
Train set: Average loss: 0.9207, Accuracy: 33743/50000 (67%)
Test set: Average loss: 1.1338, Accuracy: 6076/10000 (61%)

Epoch:  7
Train set: Average loss: 0.8532, Accuracy: 34919/50000 (70%)
Test set: Average loss: 1.1207, Accuracy: 6186/10000 (62%)

Epoch:  8
Train set: Average loss: 0.7942, Accuracy: 36031/50000 (72%)
Test set: Average loss: 1.1431, Accuracy: 6208/10000 (62%)

Epoch:  9
Train set: Average loss: 0.7302, Accuracy: 37132/50000 (74%)
Test set: Average loss: 1.1866, Accuracy: 6142/10000 (61%)

Epoch:  10
Train set: Average loss: 0.6770, Accuracy: 38075/50000 (76%)
Test set: Average loss: 1.1910, Accuracy: 6160/10000 (62%)

Best accuracy is 62.08

Training and evaluation finished

Took:  168.45 seconds
```

### Model 1

#### Code

In [ ]:
# Set parameters for Sparse Autoencoder
parser = argparse.ArgumentParser('CNN Exercise.')
parser.add_argument('--mode',
                    type=int, default=1,
                    help='Select mode between 1-3.')
parser.add_argument('--learning_rate',
                    type=float, default=0.1,
                    help='Initial learning rate.')
parser.add_argument('--num_epochs',
                    type=int,
                    default=10,
                    help='Number of epochs to run trainer.')
parser.add_argument('--batch_size',
                    type=int, default=100,
                    help='Batch size. Must divide evenly into the dataset sizes.')
parser.add_argument('--log_dir',
                    type=str,
                    default='logs',
                    help='Directory to put logging.')
                    
FLAGS = None
FLAGS, unparsed = parser.parse_known_args()

print("Mode: ", FLAGS.mode)
print("LR: ", FLAGS.learning_rate)
print("Batch size: ", FLAGS.batch_size)

start_time = time.time()
run_main(FLAGS)
end_time = time.time()

print(f'\nTook: {end_time - start_time: .2f} seconds')

Mode:  1
LR:  0.1
Batch size:  100
Torch device selected:  cuda:0
Files already downloaded and verified

Epoch:  1
Train set: Average loss: 1.8979, Accuracy: 15449/50000 (31%)
Test set: Average loss: 1.6066, Accuracy: 4233/10000 (42%)

Epoch:  2
Train set: Average loss: 1.5529, Accuracy: 22113/50000 (44%)
Test set: Average loss: 1.4281, Accuracy: 4809/10000 (48%)

Epoch:  3
Train set: Average loss: 1.3955, Accuracy: 25090/50000 (50%)
Test set: Average loss: 1.3451, Accuracy: 5150/10000 (52%)

Epoch:  4
Train set: Average loss: 1.2765, Accuracy: 27410/50000 (55%)
Test set: Average loss: 1.2252, Accuracy: 5714/10000 (57%)

Epoch:  5
Train set: Average loss: 1.1862, Accuracy: 29132/50000 (58%)
Test set: Average loss: 1.1663, Accuracy: 5844/10000 (58%)

Epoch:  6
Train set: Average loss: 1.1231, Accuracy: 30332/50000 (61%)
Test set: Average loss: 1.1686, Accuracy: 5936/10000 (59%)

Epoch:  7
Train set: Average loss: 1.0707, Accuracy: 31198/50000 (62%)
Test set: Average loss: 1.1925, Accura

#### Previous Log:

```
Mode:  1
LR:  0.1
Batch size:  100
Torch device selected:  cuda:0
Files already downloaded and verified

Epoch:  1
Train set: Average loss: 1.8628, Accuracy: 16244/50000 (32%)
Test set: Average loss: 1.5160, Accuracy: 4571/10000 (46%)

Epoch:  2
Train set: Average loss: 1.4774, Accuracy: 23481/50000 (47%)
Test set: Average loss: 1.3735, Accuracy: 5063/10000 (51%)

Epoch:  3
Train set: Average loss: 1.3291, Accuracy: 26272/50000 (53%)
Test set: Average loss: 1.2637, Accuracy: 5486/10000 (55%)

Epoch:  4
Train set: Average loss: 1.2292, Accuracy: 28281/50000 (57%)
Test set: Average loss: 1.2102, Accuracy: 5721/10000 (57%)

Epoch:  5
Train set: Average loss: 1.1502, Accuracy: 29841/50000 (60%)
Test set: Average loss: 1.1888, Accuracy: 5815/10000 (58%)

Epoch:  6
Train set: Average loss: 1.0876, Accuracy: 30897/50000 (62%)
Test set: Average loss: 1.1159, Accuracy: 6049/10000 (60%)

Epoch:  7
Train set: Average loss: 1.0319, Accuracy: 31894/50000 (64%)
Test set: Average loss: 1.0957, Accuracy: 6188/10000 (62%)

Epoch:  8
Train set: Average loss: 0.9804, Accuracy: 32788/50000 (66%)
Test set: Average loss: 1.1771, Accuracy: 6058/10000 (61%)

Epoch:  9
Train set: Average loss: 0.9422, Accuracy: 33577/50000 (67%)
Test set: Average loss: 1.0351, Accuracy: 6402/10000 (64%)

Epoch:  10
Train set: Average loss: 0.9020, Accuracy: 34253/50000 (69%)
Test set: Average loss: 1.0265, Accuracy: 6474/10000 (65%)

Best accuracy is 64.74

Training and evaluation finished
```

### Model 2

#### Code

In [ ]:
# Set parameters for Sparse Autoencoder
parser = argparse.ArgumentParser('CNN Exercise.')
parser.add_argument('--mode',
                    type=int, default=2,
                    help='Select mode between 1-3.')
parser.add_argument('--learning_rate',
                    type=float, default=0.1,
                    help='Initial learning rate.')
parser.add_argument('--num_epochs',
                    type=int,
                    default=10,
                    help='Number of epochs to run trainer.')
parser.add_argument('--batch_size',
                    type=int, default=100,
                    help='Batch size. Must divide evenly into the dataset sizes.')
parser.add_argument('--log_dir',
                    type=str,
                    default='logs',
                    help='Directory to put logging.')
                    
FLAGS = None
FLAGS, unparsed = parser.parse_known_args()

print("Mode: ", FLAGS.mode)
print("LR: ", FLAGS.learning_rate)
print("Batch size: ", FLAGS.batch_size)

start_time = time.time()
run_main(FLAGS)
end_time = time.time()

print(f'\nTook: {end_time - start_time: .2f} seconds')

Mode:  2
LR:  0.1
Batch size:  100
Torch device selected:  cuda:0
Files already downloaded and verified

Epoch:  1
Train set: Average loss: 1.7663, Accuracy: 18026/50000 (36%)
Test set: Average loss: 1.4331, Accuracy: 4908/10000 (49%)

Epoch:  2
Train set: Average loss: 1.3423, Accuracy: 26198/50000 (52%)
Test set: Average loss: 1.1926, Accuracy: 5750/10000 (58%)

Epoch:  3
Train set: Average loss: 1.1489, Accuracy: 29709/50000 (59%)
Test set: Average loss: 1.1131, Accuracy: 6072/10000 (61%)

Epoch:  4
Train set: Average loss: 1.0236, Accuracy: 31979/50000 (64%)
Test set: Average loss: 1.0527, Accuracy: 6351/10000 (64%)

Epoch:  5
Train set: Average loss: 0.9259, Accuracy: 33794/50000 (68%)
Test set: Average loss: 0.9988, Accuracy: 6562/10000 (66%)

Epoch:  6
Train set: Average loss: 0.8472, Accuracy: 35230/50000 (70%)
Test set: Average loss: 0.9607, Accuracy: 6686/10000 (67%)

Epoch:  7
Train set: Average loss: 0.7784, Accuracy: 36401/50000 (73%)
Test set: Average loss: 0.9034, Accura

#### Previous Log:

```
Mode:  2
LR:  0.1
Batch size:  100
Torch device selected:  cuda:0
Files already downloaded and verified

Epoch:  1
Train set: Average loss: 1.7852, Accuracy: 17450/50000 (35%)
Test set: Average loss: 1.4772, Accuracy: 4601/10000 (46%)

Epoch:  2
Train set: Average loss: 1.3660, Accuracy: 25537/50000 (51%)
Test set: Average loss: 1.2365, Accuracy: 5580/10000 (56%)

Epoch:  3
Train set: Average loss: 1.1770, Accuracy: 29141/50000 (58%)
Test set: Average loss: 1.1384, Accuracy: 5978/10000 (60%)

Epoch:  4
Train set: Average loss: 1.0418, Accuracy: 31632/50000 (63%)
Test set: Average loss: 1.0319, Accuracy: 6355/10000 (64%)

Epoch:  5
Train set: Average loss: 0.9432, Accuracy: 33396/50000 (67%)
Test set: Average loss: 0.9678, Accuracy: 6669/10000 (67%)

Epoch:  6
Train set: Average loss: 0.8617, Accuracy: 34971/50000 (70%)
Test set: Average loss: 0.9731, Accuracy: 6622/10000 (66%)

Epoch:  7
Train set: Average loss: 0.7902, Accuracy: 36219/50000 (72%)
Test set: Average loss: 0.9296, Accuracy: 6804/10000 (68%)

Epoch:  8
Train set: Average loss: 0.7234, Accuracy: 37331/50000 (75%)
Test set: Average loss: 0.9737, Accuracy: 6741/10000 (67%)

Epoch:  9
Train set: Average loss: 0.6577, Accuracy: 38578/50000 (77%)
Test set: Average loss: 0.9021, Accuracy: 6961/10000 (70%)

Epoch:  10
Train set: Average loss: 0.5965, Accuracy: 39515/50000 (79%)
Test set: Average loss: 0.9502, Accuracy: 6878/10000 (69%)

Best accuracy is 69.61

Training and evaluation finished

Took:  174.79 seconds
```

### Model 3

#### Code

In [ ]:
# Set parameters for Sparse Autoencoder
parser = argparse.ArgumentParser('CNN Exercise.')
parser.add_argument('--mode',
                    type=int, default=3,
                    help='Select mode between 1-3.')
parser.add_argument('--learning_rate',
                    type=float, default=0.1,
                    help='Initial learning rate.')
parser.add_argument('--num_epochs',
                    type=int,
                    default=10,
                    help='Number of epochs to run trainer.')
parser.add_argument('--batch_size',
                    type=int, default=100,
                    help='Batch size. Must divide evenly into the dataset sizes.')
parser.add_argument('--log_dir',
                    type=str,
                    default='logs',
                    help='Directory to put logging.')
                    
FLAGS = None
FLAGS, unparsed = parser.parse_known_args()

print("Mode: ", FLAGS.mode)
print("LR: ", FLAGS.learning_rate)
print("Batch size: ", FLAGS.batch_size)

start_time = time.time()
run_main(FLAGS)
end_time = time.time()

print(f'\nTook: {end_time - start_time: .2f} seconds')

Mode:  3
LR:  0.1
Batch size:  100
Torch device selected:  cuda:0
Files already downloaded and verified

Epoch:  1
Train set: Average loss: 1.9553, Accuracy: 14138/50000 (28%)
Test set: Average loss: 1.6513, Accuracy: 4064/10000 (41%)

Epoch:  2
Train set: Average loss: 1.4948, Accuracy: 22865/50000 (46%)
Test set: Average loss: 1.4566, Accuracy: 4808/10000 (48%)

Epoch:  3
Train set: Average loss: 1.3139, Accuracy: 26374/50000 (53%)
Test set: Average loss: 1.2275, Accuracy: 5598/10000 (56%)

Epoch:  4
Train set: Average loss: 1.1731, Accuracy: 28984/50000 (58%)
Test set: Average loss: 1.0988, Accuracy: 6108/10000 (61%)

Epoch:  5
Train set: Average loss: 1.0661, Accuracy: 31192/50000 (62%)
Test set: Average loss: 1.0412, Accuracy: 6322/10000 (63%)

Epoch:  6
Train set: Average loss: 0.9807, Accuracy: 32553/50000 (65%)
Test set: Average loss: 0.9792, Accuracy: 6555/10000 (66%)

Epoch:  7
Train set: Average loss: 0.9015, Accuracy: 34104/50000 (68%)
Test set: Average loss: 1.0267, Accura

#### Previous Log:

```
Mode:  3
LR:  0.1
Batch size:  100
Torch device selected:  cuda:0
Files already downloaded and verified

Epoch:  1
Train set: Average loss: 1.9598, Accuracy: 13665/50000 (27%)
Test set: Average loss: 1.5871, Accuracy: 4219/10000 (42%)

Epoch:  2
Train set: Average loss: 1.4972, Accuracy: 22794/50000 (46%)
Test set: Average loss: 1.3544, Accuracy: 5039/10000 (50%)

Epoch:  3
Train set: Average loss: 1.3026, Accuracy: 26884/50000 (54%)
Test set: Average loss: 1.2759, Accuracy: 5420/10000 (54%)

Epoch:  4
Train set: Average loss: 1.1651, Accuracy: 29222/50000 (58%)
Test set: Average loss: 1.2663, Accuracy: 5495/10000 (55%)

Epoch:  5
Train set: Average loss: 1.0573, Accuracy: 31295/50000 (63%)
Test set: Average loss: 1.0332, Accuracy: 6350/10000 (64%)

Epoch:  6
Train set: Average loss: 0.9758, Accuracy: 32919/50000 (66%)
Test set: Average loss: 1.0176, Accuracy: 6381/10000 (64%)

Epoch:  7
Train set: Average loss: 0.9032, Accuracy: 34086/50000 (68%)
Test set: Average loss: 0.9408, Accuracy: 6716/10000 (67%)

Epoch:  8
Train set: Average loss: 0.8414, Accuracy: 35149/50000 (70%)
Test set: Average loss: 0.9086, Accuracy: 6830/10000 (68%)

Epoch:  9
Train set: Average loss: 0.7875, Accuracy: 36171/50000 (72%)
Test set: Average loss: 0.8954, Accuracy: 6905/10000 (69%)

Epoch:  10
Train set: Average loss: 0.7339, Accuracy: 37091/50000 (74%)
Test set: Average loss: 0.9393, Accuracy: 6731/10000 (67%)

Best accuracy is 69.05

Training and evaluation finished

Took:  174.89 seconds
```

## Write Up

### Task 4

* The best accuracy is 69% for training and 65 for test.
* Underfitting
* ~ 169 seconds

This model appears to be underfitting for a few reasons. Both the training and testing set have high losses with low accuracies, that are close to the same performance. 

This model definitely needs more neurons and layers as it is not complex enough to learn the data properly.


### Task 5

* The best accuracy is 79% for training and 69% for test.
* Substantial overfitting
* ~ 174 seconds

This model is very obviously overfitting the data. We can see by the tenth epoch that the loss is pretty low and the accuracy is moderately high for the training set. However, the test set show a still high loss rate and a pretty low accuracy. 

Because this is the case, it's obvious that the model is overfitting quite a bit. 

The total time required to train this model was slightly more than Task 4, but that's expected since we increased the complexity quite a bit.

### Task 6

* The best accuracy is 74% (epoch 10) for training and 69% (epoch 9) for test.
* Overfitting
* ~ 174 seconds

The accuracies are closer together for both the training and test set, however there is still a gap between the two sets loss amounts. Note, it started off decent, maybe even promising, but as the epochs progressed the issues became clear as the values quickly diverged from eachother. 

That being said, it's not overfitting nearly as bad as the one from Task 5. Additional layers and neurons should be explored at this point, as well as hyperparameter tuning.

The total time to train this model was equivalent to Task 5, which is a bit suprising since the complexity of the model was amped up a little bit. I have no idea why this would be the case.